##  cloning git repository for Semeval2019Task

In [1]:
!git clone https://github.com/Semeval2019Task9/Subtask-A.git

Cloning into 'Subtask-A'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 134 (delta 7), reused 8 (delta 3), pack-reused 115
Receiving objects: 100% (134/134), 1.23 MiB | 1.66 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls


drive  sample_data  Subtask-A


In [0]:
%pycat Subtask-A/semeval-task9-baseline.py


In [0]:
keywords = ["suggest","recommend","hopefully","go for","request","it would be nice","adding","should come with","should be able","could come with", "i need" , "we need","needs", "would like to","would love to","allow","add"]


## Importing Libraries

In [0]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score


In [0]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [0]:
from nltk.stem import PorterStemmer
porterStemmer = PorterStemmer()
import re

In [0]:
import torch.nn as nn

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
stop_words=set(stopwords.words("english"))

##  Loading the test , train and predicting data

In [0]:
test_data=pd.read_csv('Subtask-A/SubtaskA_Trial_Test_Labeled.csv',encoding='ISO-8859-1')

In [0]:
train_data = pd.read_csv('Subtask-A/V1.4_Training.csv',header=None)

In [0]:
Predictin_data = pd.read_csv('Subtask-A/SubtaskA_EvaluationData.csv',header=None)

In [15]:
print(Predictin_data[:10])

      0                                                  1  2
0  9566         This would enable live traffic aware apps.  X
1  9569  Please try other formatting like bold italics ...  X
2  9576  Since computers were invented to save time I s...  X
3  9577  Allow rearranging if the user wants to change ...  X
4  9579  Add SIMD instructions for better use of ARM NE...  X
5  9580  Also using a hot swapping code generator (opti...  X
6  9587  Microsoft should seriously look into getting r...  X
7  9599  I would be extremely useful in a variety of ap...  X
8  9626  I would like to see informatin about the certi...  X
9  9628  It would be nice to see at a glance if an app ...  X


In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Trying to create glove embedding Layer

In [0]:
glove={}

In [0]:
f= open('drive/My Drive/glovedict','rb')
glove=pickle.load(f)

In [0]:
vocab={}

In [0]:

for sent in train_data[1]:
  for token in sent.split():
    if vocab.get(token) is None:
      vocab[token]=1
    else:
      vocab[token]+=1
    

In [23]:
len(vocab.keys())

20155

In [0]:
matrix_len = len(vocab)
weights_matrix = np.zeros((matrix_len, 50))
words_found = 0

for i, word in enumerate(list(vocab.keys())):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(50, ))

In [0]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [0]:

sentences=[]
for sent in train_data[1]:
  sentences.append(sent)

## Preprocessing the reviews and  creating word corpus

In [0]:
from string import punctuation

In [0]:
pun_remove=[]
for sent in sentences:
  pun=[]
  for word in sent:
    if word not in punctuation:
      pun.append(word)
  pun_remove.append(''.join(pun))
all_text = ' '.join(pun_remove)    

# # split by new lines and spaces
# reviews_split = all_text.split('\n')
# all_text = ' '.join(reviews_split)
reviews_split=pun_remove
# # create a list of words
words = all_text.split()

In [29]:
print(pun_remove)
print(words)

['Please enable removing language code from the Dev Center language history For example if you ever selected ru and ruru laguages and you published this xap to the Store then it causes Tile localization to show the enusdefault tile localization which is bad', 'Note in your csproj file there is a SupportedCultures entry like this SupportedCulturesdeDEruruRU SupportedCultures When I removed the ru language code and published my new xap version the old xap version still remains in the Store with Replaced and unpublished', 'Wich means the new version not fully replaced the old version and this causes me very serious problems 1', 'Some of my users will still receive the old xap version of my app', 'The store randomly gives the old xap or the new xap version of my app', 'My app has a WP7 version and a WP8 version XAP in the same submission', 'The wp7 xap works only on WP7 and the wp8 xap works only for WP8', 'Sometimes the Store gives the wrong wp7 xap version of my app to Windows Phone 8 us

## Building Dictionary and  mapping the sentences word to the dictionary.

In [0]:
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])

In [0]:
print(vocab_to_int.keys())

dict_keys(['the', 'to', 'a', 'and', 'is', 'in', 'I', 'of', 'for', 'it', 'be', 'that', 'on', 'app', 'with', 'have', 'this', 'not', 'would', 'you', 'an', 'as', 'can', 'or', 'Windows', 'are', 'but', 'like', 'from', 'my', 'if', 'apps', 'more', 'when', 'The', 'use', 'we', 'so', 'should', 'This', 'It', 'will', 'which', 'by', 'user', 'all', 'only', 'phone', 'could', 'do', 'using', 'at', 'way', 'one', 'add', 'no', 'has', 'there', 'support', 'new', 'Please', 'able', 'its', 'make', 'some', 'want', 'was', 'get', 'application', 'UWP', 'just', 'other', 'control', 'need', 'time', 'then', 'Phone', 'allow', 'they', 'any', 'API', 'users', 'also', 'see', 'developers', 'your', 'Microsoft', 'work', 'If', 'same', 'data', 'because', 'file', 'code', 'up', 'access', 'store', 'very', 'create', 'In', 'windows', 'does', '10', 'them', 'When', 'screen', 'me', 'developer', 'possible', 'example', 'option', 'now', 'into', 'great', 'i', 'doesnt', 'set', 'dont', 'page', 'Im', 'their', 'button', 'where', 'Feedly', 'open

##  mapping the test sentences word to the dictionary

In [0]:
test_pun_remove=[]
test_reviews_ints=[]
for sent in test_data['sentence']:
  pun=[]
  for word in sent:
    if  vocab_to_int.get(word):
      pun.append(vocab_to_int[word])
  test_reviews_ints.append(pun)
   

# # split by new lines and spaces
# reviews_split = all_text.split('\n')
# all_text = ' '.join(reviews_split)
# test_split=pun_remove
# # create a list of words

##  mapping the prediction  sentences word to the dictionary

In [0]:
predict_pun_remove=[]
predict_reviews_ints=[]
for sent in Predictin_data[1]:
  pun=[]
  for word in sent:
    if  vocab_to_int.get(word):
      pun.append(vocab_to_int[word])
  predict_reviews_ints.append(pun)

In [43]:
print(len(test_reviews_ints))
print(len(predict_reviews_ints))

592
833


##  getting the associated labels

In [34]:
len(vocab_to_int)
print(reviews_ints[0])
encoded_labels=train_data[2]

[61, 432, 1177, 255, 94, 29, 1, 469, 481, 255, 961, 149, 110, 31, 20, 929, 423, 886, 4, 2025, 2936, 4, 20, 566, 17, 1034, 2, 1, 133, 76, 10, 717, 1178, 1128, 2, 179, 1, 2937, 354, 1128, 43, 5, 470]


In [35]:
# len(test_vocab_to_int)
print(test_reviews_ints[0])
test_encoded_labels=test_data['label']

[7, 12314, 3627, 2502, 12119, 3, 3957, 115, 3627, 4245, 1278, 115, 1472, 2502, 3957, 2502, 12119, 12119, 2502, 9719, 115, 1775, 3957, 3035, 1775, 12314, 115, 3957, 3957, 115, 2502, 3627, 115, 1775, 206, 3627, 2502, 115, 3957, 2502, 3, 3627, 1618, 3, 3035, 3035, 1472, 3, 3627, 12119, 3, 1775, 12314, 1618, 3, 12119, 3, 3408, 1872, 12119, 2502, 3627, 12119, 1775, 1775, 3035, 115, 12119, 1775, 3957, 12119, 115, 1472, 12119, 1775, 115, 1775, 115, 9095, 2502, 3627, 1775]


In [44]:
# len(test_vocab_to_int)
print(predict_reviews_ints[0])
prdicted_encoded_labels=Predictin_data[2]

[9095, 115, 3957, 4114, 2502, 1872, 1775, 3627, 3, 3408, 1775, 115, 1775, 12119, 3, 115, 1472, 3, 4114, 3, 1775, 3, 3035, 3035, 3957]


## Removing outliers from the train reviews

In [37]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))

Zero-length reviews: 13
Maximum review length: 343
Number of reviews before removing outliers:  8500
Number of reviews after removing outliers:  8487


## Removing outliers from the test reviews

In [38]:
# outlier review stats
test_review_lens = Counter([len(x) for x in test_reviews_ints])
print("Zero-length reviews: {}".format(test_review_lens[0]))
print("Maximum review length: {}".format(max(test_review_lens)))
print('Number of reviews before removing outliers: ', len(test_reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
ii=0
non_zero_idx = [ii for ii, review in enumerate(test_reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
test_reviews_ints = [test_reviews_ints[ii] for ii in non_zero_idx]
test_encoded_labels = np.array([encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(test_reviews_ints))

Zero-length reviews: 0
Maximum review length: 282
Number of reviews before removing outliers:  592
Number of reviews after removing outliers:  592


In [46]:
# outlier review stats
predict_review_lens = Counter([len(x) for x in predict_reviews_ints])
print("Zero-length reviews: {}".format(predict_review_lens[0]))
print("Maximum review length: {}".format(max(predict_review_lens)))
print('Number of reviews before removing outliers: ', len(predict_reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
ii=0
non_zero_idx = [ii for ii, review in enumerate(predict_reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
predict_reviews_ints = [predict_reviews_ints[ii] for ii in non_zero_idx]
prdicted_encoded_labels = np.array([prdicted_encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(predict_reviews_ints))

Zero-length reviews: 0
Maximum review length: 288
Number of reviews before removing outliers:  833
Number of reviews after removing outliers:  833


## Padding and truncating to get the same length

In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    print()
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        print(i)
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [48]:
seq_length = 200

features = pad_features(reviews_ints, seq_length=seq_length)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
2

In [49]:
seq_length = 200

test_features = pad_features(test_reviews_ints, seq_length=seq_length)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
2

In [50]:
seq_length = 200

predict_features = pad_features(predict_reviews_ints, seq_length=seq_length)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
2

In [51]:
print(test_features.shape)
print(predict_features.shape)

(592, 200)
(833, 200)


In [52]:
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0   61  432 1177  255   94   29    1  469  481  255  961
  149  110   31   20  929  423  886    4 2025 2936    4   20  566   17
 1034    2    1  133   76   10  717 1178 1128    2  179    1 2937  354
 1128 

## Creating DataLoader for all test , train and prediction set

In [53]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_x = features
train_y = encoded_labels
test_x=test_features
test_y=test_encoded_labels
predict_x=predict_features
predict_y=prdicted_encoded_labels
print(type(features))
print(train_x.shape,train_y.shape)
train_data1 = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data1 = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
batch_size = 50
predict_data1=torch.from_numpy(predict_x)
# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data1, shuffle=True, batch_size=batch_size)

test_loader = DataLoader(test_data1, shuffle=True, batch_size=batch_size)


<class 'numpy.ndarray'>
(8487, 200) (8487,)


## Creating a Rnn model  

In [0]:
class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.num_layers =n_layers
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.hidden_size= hidden_dim
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
#         x=x.float()
        print(x.size())
        # Forward propagate RNN
        embeds = self.embedding(x)
        out,_ = self.lstm(embeds, (h0,c0))
#         out, _ = self.lstm(x, (h0, c0))  
        
        print ('out',out.size())
        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])  
        out =self.sig(out)
       # get last batch of labels
        
        # return last sigmoid output and hidden state
        return out
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

## Creating the model object

In [55]:
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(12402, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [0]:
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)


In [57]:
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [58]:
train_x.shape

(8487, 200)

In [0]:
train_y.shape
from torch.autograd import Variable

## Creating train functionalities.

In [0]:
def train():
    total_loss = 0

    for i, (feature, label) in enumerate(train_loader, 0):
#         print(feature,label)
#         print('feature',feature.size())
        output = net(feature)
#         print('out,label',len(output),len(label))
#         output=torch.FloatTensor(output)
#         print(type(output.max(axis=1)),type(label.max(axis=1)))
        loss = criterion(output, label.float())
#         loss = criterion(output.max(dim=1),label.max(dim=1))
        
  
        total_loss += loss
        print('loss',loss/(i+1.0))
        net.zero_grad()
        loss.backward()
        optimizer.step()

#         if i % 10 == 0:
#             print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
#                 time_since(start), epoch,  i *
#                 len(names), len(train_loader.dataset),
#                 100. * i * len(names) / len(train_loader.dataset),
#                 total_loss / i * len(names)))

#     return total_loss


In [0]:
for epoch in range(2):
  print(epoch)
  train()

0
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.6917, grad_fn=<DivBackward0>)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.3325, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.2106, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.1452, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.1259, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0999, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0733, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0895, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0786, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0616, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0520, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torc

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([37])) that is different to the input size (torch.Size([37, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


1
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.4529, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0877, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.1168, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0806, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0470, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0480, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0815, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0317, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0257, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out torch.Size([50, 200, 256])
loss tensor(0.0299, grad_fn=<DivBackward0>)
torch.Size([50, 200])
out to

In [0]:
# torch.save(net.state_dict(), 'drive/My Drive/LstmModel.pth')

## Loading pretrained model.

In [0]:
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
net.load_state_dict(torch.load('drive/My Drive/LstmModel.pth'))

In [0]:
# model.eval()

# Working with the test data

In [0]:
test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

#     if(train_on_gpu):
#         inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output= net(inputs)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) 
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([50, 200])
out torch.Size([50, 200, 256])
torch.Size([42, 200])
out torch.Size([42, 200, 256])
Test loss: 0.533
Test accuracy: 0.807


## Evaluating the evaluation dataset

In [62]:
pred_label=net(predict_data1)
pred_label= torch.round(pred_label.squeeze())

torch.Size([833, 200])
out torch.Size([833, 200, 256])


In [63]:
print(pred_label)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [70]:
Predictin_data[2]=list(pred_label.data.numpy().astype(int))
print(Predictin_data[:20])

       0                                                  1  2
0   9566         This would enable live traffic aware apps.  0
1   9569  Please try other formatting like bold italics ...  0
2   9576  Since computers were invented to save time I s...  0
3   9577  Allow rearranging if the user wants to change ...  0
4   9579  Add SIMD instructions for better use of ARM NE...  0
5   9580  Also using a hot swapping code generator (opti...  0
6   9587  Microsoft should seriously look into getting r...  0
7   9599  I would be extremely useful in a variety of ap...  0
8   9626  I would like to see informatin about the certi...  0
9   9628  It would be nice to see at a glance if an app ...  0
10  9630  Also I would be fun to have a record showing t...  0
11  9636  Make ProgressRing control available for Window...  1
12  9657  It would be great if the OS moved on to the se...  0
13  9660  We need for our applications on each of the mo...  0
14  9670  Provide Process Class/APIs (like that for reg

In [0]:
Predictin_data.to_csv('Ridha_Juneja.csv', sep=',',index=False,header=False)

In [0]:
class ToyNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super(self).__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [0]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [0]:
# a=one_hot_encode(list(data[2]))

In [0]:
# train_data[2]

In [0]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = torch.from_numpy(labels)
        self.list_IDs = torch.from_numpy(list_IDs)
        print(len(list_IDs))

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
#         ID = self.list_IDs[index]

        # Load data and get label
#         X = torch.load(ID)
        X=self.list_IDs[index]
#         print(X)
        y = self.labels[index]

        return X, y

In [0]:
# X=list(train_data[1])
# Y=list(train_data[2])
# X=np.array(X)
# Y=np.array(Y)
# train_dataset = Dataset(X,Y)
# train_loader = data.DataLoader(dataset=train_dataset,
# batch_size=16)